## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
import numpy as np

A = np.random.randint(0,1000,size=(1000000,))

In [ ]:
import time
import pandas as pd
from numba import jit,njit


def f1(x):
  start = time.time()
  acc,cnt = 0,0
  for x in A:
    acc += (x+100)
    cnt += 1
  a = acc/cnt
  end = time.time()
  print("Время выполнения ",end-start)
  return a
  

def f2(x):
  start = time.time()
  acc = 0
  for x in A:
    acc += (x+100)
  a = acc/(len(A))
  end = time.time()
  print("Время выполнения ",end-start)
  return a

@njit
def f3(x):
  #start = time.time()
  acc = 0
  for x in A:
    acc += x
  a = acc/len(A)+100
  #end = time.time()
  #print("Время выполнения ",end-start)
  return a

print(f1(A))
print(f2(A))
print(f3(A))




Время выполнения  0.34353208541870117
599.402038
Время выполнения  0.2887997627258301
599.402038
599.402038


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
import string 
N = 2_000_000
#, columns=[f'col{i}' for i in range(4)]
df = pd.DataFrame(np.random.randn(N,4))
df['key'] = np.random.choice(list(string.ascii_letters.lower()),N, replace = True)
s = []
for i in range(N):
  df.key[i] = 'a'
  s.append(df.[i])
for j in range(N):
  df.key[j] = 'b'
  s.append(df[j])
for k in range(N):
  df.key[k] = 'c'
  s.append(df[k])
for p in range(N):
  df.key[p] = 'd'
  s.append(df[p])
for q in range(N):
  df.key[q] = 'e'
  s.append(df[q])

    

print(s)


<ipython-input-43-f8c558e3ddbb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.key[i] = 'a'


KeyError: ignored

## Лабораторная работа 3

In [ ]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 10.1 MB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [ ]:
import numpy as np
import pandas as pd
recipes = pd.read_csv('recipes_sample.csv')
recipes = pd.DataFrame(recipes)
recipes[:3]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [ ]:
reviews = pd.read_csv('reviews_sample.csv')
reviews = pd.DataFrame(reviews)
reviews[:3]

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
5,910362,35106,31322,2003-01-03,4,I forgot to add skim milk but it still tasted ...
6,212649,404333,199579,2006-12-10,5,"Made this for dinner it was so excellent, fina..."
7,815389,162888,16067,2005-12-09,5,"When I snapped the picture, I forgot to review..."
8,642377,89831,33715,2007-07-03,5,This was good combination of flavors but I wil...
9,1023302,308434,11252,2008-12-14,5,Oh Bergy! These wonderful little cakes are aw...


In [ ]:
#A
#Реализуйте несколько вариантов функции подсчета среднего значения столбца rating из таблицы reviews для отзывов, оставленных в 2010 году.
#A. С использованием метода DataFrame.iterrows исходной таблицы;

#for row in reviews[:5].itertuples():
   # print(row)
#from numba import njit
#import numpy as np
import time
a = []
c = 0
start = time.time()
for index, row in reviews.iterrows():
  if '2010' in row['date']:   
    a.append(row['rating'])
    c = c + row['rating']
r = c/len(a)
end = time.time()
print(r)
print("Время выполнения ",end-start)


4.4544402182900615
Время выполнения  3.8714733123779297


In [ ]:
#Б
dat = []
for index, row in reviews.iterrows():
  if '2010' in row['date']:
    dat.append(row['date'])

reviews2010 = pd.DataFrame({'rating':a,'date': dat})
start = time.time()
cow = 0
coke = []
for index, row in reviews2010.iterrows():
  cow += row['rating']
  coke.append(row['rating'])
coke = cow/len(coke)
print(coke)
end = time.time()
print("Время выполнения ",end-start)
#print(a)

4.4544402182900615
Время выполнения  0.3945624828338623


In [ ]:
#reviews2010 = pd.DataFrame({'rating':a,'date': dat})
#reviews2010[:7]
cow = 0
coke = []
for index, row in reviews2010.iterrows():
  cow += row['rating']
  coke.append(row['rating'])
coke = cow/len(coke)
print(coke)


4.4544402182900615


In [ ]:
#В
# С использованием метода Series.mean.
from numba import jit
import pandas as pd


start = time.time()
print(reviews2010['rating'].mean(axis=0))
end = time.time()
print("Время выполнения ",end-start)

4.4544402182900615
Время выполнения  0.001608133316040039


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f f1 f1(A)

Время выполнения  3.1431379318237305


In [ ]:
%lprun -f f2 f2(A)

Время выполнения  1.8968160152435303


In [ ]:
%lprun -f f3 f3(A)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
start = time.time()
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
end = time.time()
print("Время выполнения ",end-start)

Время выполнения  0.00015115737915039062


In [ ]:
start = time.time()
def get_word_reviews_count_new(df):
  word_reviews = {}
  for row in df.dropna(subset=['review'])['review'].str.split(' '):
    for word in row:
      if word in word_reviews:
        word_reviews[word] += 1
  return word_reviews
end = time.time()
print("Время выполнения ",end-start)     


Время выполнения  0.0001366138458251953


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4